In [9]:
import hmac
import time
import hashlib
import requests as rqst
import math
import pandas as pd
from urllib.parse import urlencode

In [10]:
from secrets_production import API_KEY, API_SECRET

In [17]:
KEY = API_KEY
SECRET = API_SECRET
BASE_URL = 'https://api.binance.com' # production base url
#BASE_URL = 'https://testnet.binance.vision' # testnet base url

coin_gecko_api_url = 'https://api.coingecko.com/api/v3/coins/markets'
coins_info = rqst.get(f'{coin_gecko_api_url}?vs_currency=usd&order=market_cap_desc&per_page=250&page=1&sparkline=false')
coins_info.json()
# https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&order=market_cap_desc&per_page=100&page=1&sparkline=false
coin_gecko_list = coins_info.json()

In [4]:
''' ======  begin of functions, you don't need to touch ====== '''
def hashing(query_string):
    return hmac.new(SECRET.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest()

def get_timestamp():
    return int(time.time() * 1000)


def dispatch_request(http_method):
    session = requests.Session()
    session.headers.update({
        'Content-Type': 'application/json;charset=utf-8',
        'X-MBX-APIKEY': KEY
    })
    return {
        'GET': session.get,
        'DELETE': session.delete,
        'PUT': session.put,
        'POST': session.post,
    }.get(http_method, 'GET')

# used for sending request requires the signature
def send_signed_request(http_method, url_path, payload={}):
    query_string = urlencode(payload, True)
    if query_string:
        query_string = "{}&timestamp={}".format(query_string, get_timestamp())
    else:
        query_string = 'timestamp={}'.format(get_timestamp())

    url = BASE_URL + url_path + '?' + query_string + '&signature=' + hashing(query_string)
    print("{} {}".format(http_method, url))
    params = {'url': url, 'params': {}}
    response = dispatch_request(http_method)(**params)
    return response.json()

# used for sending public data request
def send_public_request(url_path, payload={}):
    query_string = urlencode(payload, True)
    url = BASE_URL + url_path
    if query_string:
        url = url + '?' + query_string
    print("{}".format(url))
    response = dispatch_request('GET')(url=url)
    return response.json()

# used for rounding decimals of orders amount based in Binance calculations 
def round_decimals_down(number:float, decimals:int=2):
    """    Returns a value rounded down to a specific number of decimal places.    """
    if not isinstance(decimals, int):
        raise TypeError("decimal places must be an integer")
    elif decimals < 0:
        raise ValueError("decimal places has to be 0 or more")
    elif decimals == 0:
        return math.floor(number)

    factor = 10 ** decimals
    return math.floor(number * factor) / factor
''' ======  end of functions ====== '''

' ======  end of functions ====== '

In [42]:
### USER_DATA endpoints, call send_signed_request #####
# get account informtion
# if you can see the account details, then the API key/secret is correct
response = send_signed_request('GET', '/api/v3/account')

GET https://api.binance.com/api/v3/account?timestamp=1632717546539&signature=3ad7bcd0349ec2096bba3cc742d97fe9b6f7b9e24dab3e026f8051a88c4eae92


In [43]:
spot_assets = response['balances']
spot_assets

[{'asset': 'BTC', 'free': '0.03328997', 'locked': '0.00000000'},
 {'asset': 'LTC', 'free': '0.00000000', 'locked': '0.00000000'},
 {'asset': 'ETH', 'free': '0.00000000', 'locked': '0.00000000'},
 {'asset': 'NEO', 'free': '0.00000000', 'locked': '0.00000000'},
 {'asset': 'BNB', 'free': '0.00205491', 'locked': '0.00000000'},
 {'asset': 'QTUM', 'free': '0.00000000', 'locked': '0.00000000'},
 {'asset': 'EOS', 'free': '0.00000000', 'locked': '0.00000000'},
 {'asset': 'SNT', 'free': '0.00000000', 'locked': '0.00000000'},
 {'asset': 'BNT', 'free': '0.00000000', 'locked': '0.00000000'},
 {'asset': 'GAS', 'free': '0.00000000', 'locked': '0.00000000'},
 {'asset': 'BCC', 'free': '0.00000000', 'locked': '0.00000000'},
 {'asset': 'USDT', 'free': '169.54633932', 'locked': '0.00000000'},
 {'asset': 'HSR', 'free': '0.00000000', 'locked': '0.00000000'},
 {'asset': 'OAX', 'free': '0.00000000', 'locked': '0.00000000'},
 {'asset': 'DNT', 'free': '0.00000000', 'locked': '0.00000000'},
 {'asset': 'MCO', 'fr

In [44]:
my_columns = ['Ticker', 'Balance', 'Locked', 'Price', 'Value in USD']
assets_dataframe = pd.DataFrame(columns = my_columns)
assets_dataframe

,Ticker,Balance,Locked,Price,Value in USD


In [45]:
for assets in spot_assets:
    ticker = assets['asset']
    locked = float(assets['locked'])
    free = float(assets['free']) 
    i_total_asset = free + locked
    if i_total_asset > 0.0:
        assets_dataframe = assets_dataframe.append(
            pd.Series(
            [
                ticker,
                i_total_asset,
                locked,
                'N/A',
                'N/A'
            ],        
            index = my_columns),    
            ignore_index = True
        )        
assets_dataframe

,Ticker,Balance,Locked,Price,Value in USD
0,BTC,0.033290,0.0,N/A,N/A
1,BNB,0.002055,0.0,N/A,N/A
2,USDT,169.546339,0.0,N/A,N/A
3,FET,1001.997000,0.0,N/A,N/A
4,ONE,58.200000,0.0,N/A,N/A
5,CHZ,100.000000,0.0,N/A,N/A
6,ALGO,79.970000,0.0,N/A,N/A
7,BUSD,1618.205565,0.0,N/A,N/A
8,XTZ,8.000000,0.0,N/A,N/A
9,DOT,10.000000,0.0,N/A,N/A


#### Append price and Value in USD to the Dataframe

In [47]:
for i in range(0, len(assets_dataframe)):
    for coins in coin_gecko_list:
        price = float(coins ['current_price'])
        if assets_dataframe.loc[i, 'Ticker'].upper() == coins['symbol'].upper():
            assets_dataframe.loc[i, 'Value in USD'] = price * assets_dataframe.loc[i, 'Balance']
            assets_dataframe.loc[i, 'Price'] = price
assets_dataframe

,Ticker,Balance,Locked,Price,Value in USD
0,BTC,0.033290,0.0,44130.0,1469.086376
1,BNB,0.002055,0.0,353.56,0.726534
2,USDT,169.546339,0.0,1.0,169.546339
3,FET,1001.997000,0.0,0.806167,807.776915
4,ONE,58.200000,0.0,0.168603,9.812695
5,CHZ,100.000000,0.0,0.268492,26.8492
6,ALGO,79.970000,0.0,1.83,146.3451
7,BUSD,1618.205565,0.0,1.0,1618.205565
8,XTZ,8.000000,0.0,6.48,51.84
9,DOT,10.000000,0.0,29.73,297.3


#### Organize dataframe by Assets Value 

In [48]:
assets_dataframe.sort_values('Value in USD', ascending = False, inplace = True)
assets_dataframe

,Ticker,Balance,Locked,Price,Value in USD
7,BUSD,1618.205565,0.0,1.0,1618.205565
0,BTC,0.033290,0.0,44130.0,1469.086376
3,FET,1001.997000,0.0,0.806167,807.776915
9,DOT,10.000000,0.0,29.73,297.3
2,USDT,169.546339,0.0,1.0,169.546339
6,ALGO,79.970000,0.0,1.83,146.3451
11,EGLD,0.400000,0.0,230.97,92.388
14,AR,2.000000,0.0,45.99,91.98
12,KSM,0.255900,0.0,333.04,85.224936
10,LUNA,1.998000,0.0,36.85,73.6263
